# Pasul 2 – Colectarea și popularea bazei de date (Date WDI)

**Obiectiv:** Descărcarea și normalizarea indicatorilor socio-economici din
World Development Indicators (WDI) pentru un set de țări și ani, în format
tabelar (long), pregătit pentru încărcarea în baza de date și combinarea
ulterioară cu datele EIA.

## Context și unitatea de analiză

Indicatorii WDI oferă informații socio-economice relevante (PIB, consum
energetic, acces la electricitate, energie regenerabilă), necesare pentru
analiza relației dintre dezvoltarea economică și tranziția energetică.

Unitatea de analiză utilizată este **țară × an**, pentru a asigura
compatibilitatea cu indicatorii energetici proveniți din EIA.

## Indicatori, țări și perioadă analizată

**Indicatori WDI selectați:**
- PIB per capita – `NY.GDP.PCAP.CD`
- Consum de energie per capita – `EG.USE.PCAP.KG.OE`
- Pondere energie regenerabilă – `EG.FEC.RNEW.ZS`
- Acces la electricitate – `EG.ELC.ACCS.ZS`

**Țări analizate:** România, Bulgaria, Ungaria, Polonia, Cehia, Slovacia  
**Coduri ISO3:** ROU, BGR, HUN, POL, CZE, SVK  

**Perioadă analizată:** 2020–2023 (în funcție de disponibilitatea datelor WDI)

## Import librării

In [1]:
import requests
import pandas as pd

## Definire indicatori și țări

In [2]:
# Indicatorii WDI selectați
WDI_INDICATORS = {
    "gdp_per_capita": "NY.GDP.PCAP.CD",
    "energy_use_per_capita": "EG.USE.PCAP.KG.OE",
    "renewables_share": "EG.FEC.RNEW.ZS",
    "electricity_access": "EG.ELC.ACCS.ZS",
}

# Țările analizate (ISO3)
COUNTRIES = ["ROU", "BGR", "HUN", "POL", "CZE", "SVK"]

# Anii analizați
YEARS = [2020, 2021, 2022, 2023]

## Descărcarea datelor din WDI API

Pentru fiecare indicator WDI, datele sunt descărcate prin API-ul World Bank,
în format JSON. Funcția de mai jos returnează răspunsul brut, care va fi ulterior
normalizat într-un DataFrame pandas.

In [3]:
def get_wdi_indicator(indicator_code):
    """
    Descarcă datele WDI pentru un indicator dat.
    Returnează răspunsul JSON al API-ului World Bank.
    """
    url = f"https://api.worldbank.org/v2/country/all/indicator/{indicator_code}"
    params = {
        "format": "json",
        "per_page": 20000
    }
    response = requests.get(url, params=params)
    response.raise_for_status()
    return response.json()

## Descărcarea tuturor indicatorilor WDI

Datele sunt descărcate separat pentru fiecare indicator și stocate temporar
pentru procesarea ulterioară.

In [4]:
wdi_raw = {}

for name, code in WDI_INDICATORS.items():
    print(f"Downloading {name} ({code}) ...")
    wdi_raw[name] = get_wdi_indicator(code)

## Normalizare: JSON → DataFrame (format long)

Datele JSON sunt transformate într-un tabel „tidy/long”, unde fiecare rând
reprezintă o observație unică (țară, an, indicator).

In [5]:
dfs = []

for indicator_name, json_obj in wdi_raw.items():
    rows = json_obj[1]  # lista de observații
    df = pd.DataFrame(rows)

    # filtrare țări și ani
    df = df[df["countryiso3code"].isin(COUNTRIES)]
    df = df[df["date"].astype(int).isin(YEARS)]

    # curățare câmpuri
    df["countryRegionId"] = df["countryiso3code"]
    df["year"] = df["date"].astype(int)
    df["indicator"] = indicator_name
    df["value"] = pd.to_numeric(df["value"], errors="coerce")

    dfs.append(df[["countryRegionId", "year", "indicator", "value"]])

# concatenare finală
wdi_long = pd.concat(dfs, ignore_index=True)

wdi_long.head(10)

,countryRegionId,year,indicator,value
0,BGR,2023,gdp_per_capita,15883.861855
1,BGR,2022,gdp_per_capita,13999.671920
2,BGR,2021,gdp_per_capita,12972.217336
3,BGR,2020,gdp_per_capita,10769.947570
4,CZE,2023,gdp_per_capita,31591.056822
5,CZE,2022,gdp_per_capita,28282.223672
6,CZE,2021,gdp_per_capita,27696.461953
7,CZE,2020,gdp_per_capita,23472.891545
8,HUN,2023,gdp_per_capita,22312.179029
9,HUN,2022,gdp_per_capita,18483.636687


## Verificări rapide

Sunt verificate:
- dimensiunea dataset-ului
- anii disponibili
- țările incluse
- existența valorilor lipsă sau a duplicatelor

In [6]:
print("Shape:", wdi_long.shape)
print("Ani disponibili:", sorted(wdi_long["year"].unique()))
print("Țări:", sorted(wdi_long["countryRegionId"].unique()))
print("Indicatori:", sorted(wdi_long["indicator"].unique()))

# duplicate pe cheia principală
dups = wdi_long.duplicated(
    subset=["countryRegionId", "year", "indicator"]
).sum()
print("Duplicate (country, year, indicator):", dups)

# valori lipsă
print("\nValori lipsă per coloană:\n", wdi_long.isna().sum())

Shape: (96, 4)
Ani disponibili: [np.int64(2020), np.int64(2021), np.int64(2022), np.int64(2023)]
Țări: ['BGR', 'CZE', 'HUN', 'POL', 'ROU', 'SVK']
Indicatori: ['electricity_access', 'energy_use_per_capita', 'gdp_per_capita', 'renewables_share']
Duplicate (country, year, indicator): 0

Valori lipsă per coloană:
 countryRegionId     0
year                0
indicator           0
value              12
dtype: int64


## Observații și limitări

Pentru unii indicatori (ex. ponderea energiilor regenerabile) pot exista valori
lipsă pentru anumite țări sau ani. Aceste valori sunt păstrate ca NaN în această
etapă.

Completarea valorilor lipsă, agregarea și transformările statistice vor fi
realizate în etapa următoare a proiectului (Pasul 3).

## Salvarea datelor

Dataset-ul final este salvat în format CSV, în structură (normalizată),
pregătit pentru încărcarea în baza de date și combinarea ulterioară cu datele EIA.

In [ ]:
wdi_long.to_csv("02_wdi_data_student2.csv", index=False)
print("Salvat: 02_wdi_data_student2.csv")